<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
import functools
import heapq
from queue import PriorityQueue
import copy
import numpy as np  # Group imports at the top

# Define constants in UPPER_SNAKE_CASE
SOLUTION_8_PUZZLE = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
]
POSSIBLE_MOVEMENTS = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

In [2]:
class PuzzleNode:
    """
    Represents a node in the A* search for solving the n^2-1 puzzle.

    Each node encapsulates a puzzle state and related search information
    like path cost, heuristic estimate, and parent node for path reconstruction.
    """

    def __init__(
        self, puzzle_state, parent_node=None, path_cost=0, heuristic_function=None
    ):
        """
        Initializes a PuzzleNode.

        Args:
            puzzle_state (list[list[int]]): The current configuration of the puzzle.
            parent_node (Optional[PuzzleNode]): The node from which this node was reached.
            path_cost (int): The cost to reach this node from the start state (g value).
            heuristic_function (Optional[Callable]): Heuristic function to estimate cost to goal (h function).
        """
        self.state = puzzle_state
        self.parent_node = parent_node
        self.path_cost_g = path_cost
        self.heuristic_function = heuristic_function
        self.heuristic_value_h = (
            heuristic_function(self.state) if heuristic_function else 0
        )  # Calculate h value
        self.total_cost_f = (
            self.path_cost_g + self.heuristic_value_h
        )  # Calculate f = g + h

    def get_successors(self):
        """
        Generates child nodes representing valid next states by expanding the current node.

        Returns:
            list[PuzzleNode]: A list of PuzzleNode instances representing reachable next states.
        """
        children_nodes = []
        empty_tile_location = (
            self._find_empty_tile_position()
        )  # Use helper method for clarity

        for move_row, move_col in POSSIBLE_MOVEMENTS:
            next_row_index, next_col_index = (
                empty_tile_location[0] + move_row,
                empty_tile_location[1] + move_col,
            )
            if self._is_valid_move(
                next_row_index, next_col_index
            ):  # Use helper method for move validation
                next_puzzle_state = self._create_next_state(
                    empty_tile_location, next_row_index, next_col_index
                )
                children_nodes.append(
                    PuzzleNode(
                        next_puzzle_state,
                        self,
                        self.path_cost_g + 1,
                        self.heuristic_function,
                    )
                )
        return children_nodes

    def _find_empty_tile_position(self):
        """Helper method to locate the position of the empty tile (0)."""
        for row_index in range(len(self.state)):
            for col_index in range(len(self.state[row_index])):
                if self.state[row_index][col_index] == 0:
                    return row_index, col_index
        return None  # Should not happen in valid puzzles

    def _is_valid_move(self, row_index, col_index):
        """Helper method to check if a move is within the puzzle boundaries."""
        return 0 <= row_index < len(self.state) and 0 <= col_index < len(self.state[0])

    def _create_next_state(self, empty_tile_location, next_row_index, next_col_index):
        """Helper method to create a new state by moving the empty tile."""
        next_puzzle_state = [list(row) for row in self.state]  # Create deep copy
        (
            next_puzzle_state[empty_tile_location[0]][empty_tile_location[1]],
            next_puzzle_state[next_row_index][next_col_index],
        ) = (next_puzzle_state[next_row_index][next_col_index], 0)
        return next_puzzle_state

    def __lt__(self, other_node):
        """
        Less than comparison based on total cost (f value), used by PriorityQueue.

        Args:
            other_node (PuzzleNode): The node to compare with.

        Returns:
            bool: True if this node's f value is less than the other node's, False otherwise.
        """
        return self.total_cost_f < other_node.total_cost_f

    def __str__(self):
        """
        Returns a string representation of the puzzle state, formatted as a grid.

        Returns:
            str: Grid-like string representation of the puzzle state.
        """
        state_str = ""
        for row in self.state:
            state_str += (
                "| "
                + " | ".join(f"{tile:2}" if tile != 0 else "  " for tile in row)
                + " |\n"
            )
        return state_str

In [3]:
# Test case for the PuzzleNode class
def test_puzzle_node():
    """
    Test function to demonstrate the functionality of the PuzzleNode class.
    """
    initial_state = [
        [1, 2, 3],
        [4, 0, 5],
        [6, 7, 8],
    ]
    puzzle_node = PuzzleNode(initial_state)
    print("Initial Puzzle State:")
    print(puzzle_node)

    successors = puzzle_node.get_successors()
    print("Successor States:")
    for successor in successors:
        print(successor)


test_puzzle_node()

Initial Puzzle State:
|  1 |  2 |  3 |
|  4 |    |  5 |
|  6 |  7 |  8 |

Successor States:
|  1 |  2 |  3 |
|  4 |  5 |    |
|  6 |  7 |  8 |

|  1 |  2 |  3 |
|    |  4 |  5 |
|  6 |  7 |  8 |

|  1 |  2 |  3 |
|  4 |  7 |  5 |
|  6 |    |  8 |

|  1 |    |  3 |
|  4 |  2 |  5 |
|  6 |  7 |  8 |



<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [4]:
def to_immutable_state(state_list):
    """
    Converts a 2D list puzzle state to a hashable tuple of tuples.

    This is necessary for using puzzle states as keys in sets or dictionaries (e.g., for memoization or explored sets).

    Args:
        state_list (list[list[int]]): The 2D list representing the puzzle state.

    Returns:
        tuple[tuple[int]]: An immutable tuple of tuples representing the same state.
    """
    return tuple(tuple(row) for row in state_list)


def memoize_heuristic(func):
    """
    Decorator to memoize heuristic function results, improving performance by caching.

    This decorator stores and reuses the results of heuristic function calls based on input arguments,
    avoiding redundant computations for previously evaluated puzzle states.
    """
    memo_cache = {}

    def cached_function(*args, **kwargs):
        """Wrapper function with memoization logic."""
        hashable_args = []
        for arg in args:
            if isinstance(arg, list):
                if all(
                    isinstance(item, list) for item in arg if isinstance(item, list)
                ):
                    hashable_arg = tuple(tuple(r) for r in arg)
                else:
                    hashable_arg = tuple(arg)
            else:
                hashable_arg = arg
            hashable_args.append(hashable_arg)

        cache_key = (
            tuple(hashable_args),
            frozenset(kwargs.items()) if kwargs else None,
        )
        if cache_key not in memo_cache:
            memo_cache[cache_key] = func(*args, **kwargs)
        return memo_cache[cache_key]

    return cached_function


def create_goal_state(grid_dimension):
    """
    Generates the canonical goal state for an n x n sliding puzzle.

    Tiles are arranged in ascending order from 0 to n²-1, with 0 representing the blank tile and located at the top-left.

    Args:
        grid_dimension (int): The size of the puzzle grid (n x n).

    Returns:
        list[list[int]]: The generated goal state as a 2D list.
    """
    goal_state = [[0 for _ in range(grid_dimension)] for _ in range(grid_dimension)]
    tile_value = 0
    for row_index in range(grid_dimension):
        for col_index in range(grid_dimension):
            goal_state[row_index][col_index] = tile_value
            tile_value += 1
    return goal_state


def get_goal_positions(grid_dimension):
    """
    Creates a lookup dictionary for tile positions in the goal state, optimizing heuristic calculations.

    This function pre-calculates and stores the goal position (row and column) for each tile value,
    allowing for efficient O(1) lookup during heuristic computation, particularly for Manhattan distance.

    Args:
        grid_dimension (int): The size of the puzzle grid (n x n).

    Returns:
        dict[int, tuple[int, int]]: A dictionary mapping each tile value to its (row, column) position in the goal state.
    """
    solved_state = create_goal_state(grid_dimension)
    tile_locations = {}
    for row_index in range(grid_dimension):
        for col_index in range(grid_dimension):
            tile_locations[solved_state[row_index][col_index]] = (row_index, col_index)
    return tile_locations


@memoize_heuristic
def h1(puzzle_config):
    """
    Calculates the Misplaced Tiles heuristic value for a given puzzle state.

    This heuristic counts the number of tiles that are not in their goal positions. It is admissible because
    each misplaced tile requires at least one move to reach its correct position.

    Args:
        puzzle_config (list[list[int]]): The current puzzle state.

    Returns:
        int: The number of misplaced tiles (excluding the blank tile).
    """
    solved_config = create_goal_state(len(puzzle_config))
    misplaced_count = 0
    for row_index in range(len(puzzle_config)):
        for col_index in range(len(puzzle_config[row_index])):
            if (
                puzzle_config[row_index][col_index]
                != solved_config[row_index][col_index]
                and puzzle_config[row_index][col_index] != 0
            ):
                misplaced_count += 1
    return misplaced_count


@memoize_heuristic
def h2(puzzle_config):
    """
    Calculates the Manhattan Distance heuristic value for a given puzzle state.

    This heuristic sums the Manhattan distances (L1 distance) of each tile from its current position
    to its goal position. It is admissible because each tile must move at least its Manhattan distance
    to reach its correct position in the goal state.

    Args:
        puzzle_config (list[list[int]]): The current puzzle state.

    Returns:
        int: The total Manhattan distance for all tiles (excluding the blank tile).
    """
    grid_dimension = len(puzzle_config)
    target_tile_positions = get_goal_positions(grid_dimension)
    total_distance = 0
    for row_index in range(grid_dimension):
        for col_index in range(grid_dimension):
            tile_value = puzzle_config[row_index][col_index]
            if tile_value != 0:  # Ignore blank tile
                goal_row_pos, goal_col_pos = target_tile_positions[tile_value]
                total_distance += abs(row_index - goal_row_pos) + abs(
                    col_index - goal_col_pos
                )
    return total_distance


def h3(puzzle_config):
    """
    Implements an enhanced heuristic combining pattern database and linear conflict detection.

    For 3x3 puzzles, it attempts to use a pattern database for more accurate estimates.
    For other puzzles or when the state is not in the database, it falls back to Manhattan distance
    enhanced with linear conflict detection. Linear conflicts account for tiles in their correct row or column
    but in reversed order, requiring additional moves.

    Args:
        puzzle_config (list[list[int]]): The current puzzle state.

    Returns:
        int: A heuristic estimate that is generally more informed than Manhattan distance alone.
    """
    from collections import defaultdict

    current_state_tuple = to_immutable_state(puzzle_config)

    if len(puzzle_config) == 3:
        if "PATTERN_DATABASE" not in globals():
            global PATTERN_DATABASE
            PATTERN_DATABASE = defaultdict(lambda: float("inf"))
            solved_state_tuple = to_immutable_state(create_goal_state(3))
            PATTERN_DATABASE[solved_state_tuple] = 0

            import queue

            bfs_queue = queue.Queue()
            bfs_queue.put((solved_state_tuple, 0))

            while not bfs_queue.empty():
                state_tup, distance = bfs_queue.get()
                if distance > PATTERN_DATABASE[state_tup]:
                    continue

                state_list_repr = [list(row) for row in state_tup]
                blank_position = next(
                    (
                        (r_idx, c_idx)
                        for r_idx, row in enumerate(state_list_repr)
                        for c_idx, val in enumerate(row)
                        if val == 0
                    ),
                    None,
                )

                for dr, dc in POSSIBLE_MOVEMENTS:
                    next_r, next_c = blank_position[0] + dr, blank_position[1] + dc
                    if 0 <= next_r < 3 and 0 <= next_c < 3:
                        next_state = [row[:] for row in state_list_repr]
                        (
                            next_state[blank_position[0]][blank_position[1]],
                            next_state[next_r][next_c],
                        ) = (
                            next_state[next_r][next_c],
                            next_state[blank_position[0]][blank_position[1]],
                        )
                        next_state_tuple = tuple(tuple(row) for row in next_state)

                        if distance + 1 < PATTERN_DATABASE[next_state_tuple]:
                            PATTERN_DATABASE[next_state_tuple] = distance + 1
                            bfs_queue.put((next_state_tuple, distance + 1))

        if current_state_tuple in PATTERN_DATABASE:
            return PATTERN_DATABASE[current_state_tuple]

    manhattan_h = h2(puzzle_config)
    linear_conflicts_count = 0
    grid_dimension = len(puzzle_config)
    solved_state = create_goal_state(grid_dimension)

    # Detect linear conflicts in rows
    for row_index in range(grid_dimension):
        row_tiles_in_place = []
        for col_index in range(grid_dimension):
            tile = puzzle_config[row_index][col_index]
            if tile != 0 and tile in solved_state[row_index]:
                row_tiles_in_place.append((tile, col_index))
        row_tiles_in_place.sort(
            key=lambda x: solved_state[row_index].index(x[0])
        )  # Sort by goal column

        for i in range(len(row_tiles_in_place)):
            for j in range(i + 1, len(row_tiles_in_place)):
                if (
                    row_tiles_in_place[i][1] > row_tiles_in_place[j][1]
                ):  # Conflict if in wrong order
                    linear_conflicts_count += 2

    # Detect linear conflicts in columns
    for col_index in range(grid_dimension):
        col_tiles_in_place = []
        for row_index in range(grid_dimension):
            tile = puzzle_config[row_index][col_index]
            goal_col = [solved_state[r][col_index] for r in range(grid_dimension)]
            if tile != 0 and tile in goal_col:
                col_tiles_in_place.append((tile, row_index))
        col_tiles_in_place.sort(
            key=lambda x: [
                solved_state[r][col_index] for r in range(grid_dimension)
            ].index(x[0])
        )  # Sort by goal row

        for i in range(len(col_tiles_in_place)):
            for j in range(i + 1, len(col_tiles_in_place)):
                if (
                    col_tiles_in_place[i][1] > col_tiles_in_place[j][1]
                ):  # Conflict if in wrong order
                    linear_conflicts_count += 2

    return manhattan_h + linear_conflicts_count


# List of heuristic functions available for the A* search algorithm
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [5]:
def get_empty_tile_pos(puzzle_state):
    """
    Finds the row and column index of the empty tile (represented by 0) in the puzzle state.

    This function iterates through the puzzle state to locate the tile with value 0,
    which represents the empty space in the sliding puzzle.

    Args:
        puzzle_state (list[list[int]]): The current state of the puzzle, represented as a 2D list.

    Returns:
        Optional[tuple[int, int]]: A tuple (row, column) representing the position of the empty tile.
                                  Returns None if the empty tile (0) is not found, though this should not occur in a valid puzzle state.
    """
    for row_index in range(len(puzzle_state)):
        for col_index in range(len(puzzle_state[row_index])):
            if puzzle_state[row_index][col_index] == 0:
                return row_index, col_index
    return (
        None  # Return None if blank tile not found (should not happen in valid puzzles)
    )


def is_valid_puzzle_state(puzzle_state):
    """
    Checks if the given puzzle state is valid for an n x n sliding puzzle.

    Validity criteria include:
    1. The state must be represented as an n x n grid (list of lists), where n is at least 3.
    2. The grid must contain all numbers from 0 to n²-1 exactly once.

    Args:
        puzzle_state (list[list[int]]): The puzzle state to be validated.

    Returns:
        bool: True if the puzzle state is valid, False otherwise.
    """
    if len(puzzle_state) < 3:  # Rule 1: Minimum size check (n >= 3)
        return False
    for row in puzzle_state:  # Rule 1: Square grid check (n x n)
        if len(row) != len(puzzle_state):
            return False

    grid_dimension = len(puzzle_state)
    expected_tiles = set(
        range(grid_dimension * grid_dimension)
    )  # Set of expected tile values
    current_tiles = set()
    for row in puzzle_state:
        for tile in row:
            if not isinstance(tile, int) or tile not in range(
                grid_dimension * grid_dimension
            ):  # Rule 2: Type and range check (0 to n²-1)
                return False
            if tile in current_tiles:  # Rule 2: Check for duplicate tiles
                return False
            current_tiles.add(tile)

    return (
        current_tiles == expected_tiles
    )  # Rule 2: Ensure all expected tiles are present exactly once


def count_inversions(puzzle_state):
    """
    Counts the number of inversions in a given puzzle state.

    An inversion is a pair of tiles (i, j) where tile i appears before tile j in the flattened puzzle state,
    but the value of tile i is greater than the value of tile j. The blank tile (0) is not considered in inversion counts.

    Args:
        puzzle_state (list[list[int]]): The puzzle state for which inversions are to be counted.

    Returns:
        int: The total number of inversions in the puzzle state.
    """
    flat_puzzle = [
        puzzle_state[row_index][col_index]
        for row_index in range(len(puzzle_state))
        for col_index in range(len(puzzle_state[row_index]))
        if puzzle_state[row_index][col_index]
        != 0  # Exclude the blank tile from inversion calculation
    ]
    inversion_count = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if (
                flat_puzzle[i] > flat_puzzle[j]
            ):  # Condition for an inversion: tile i > tile j but appears before j
                inversion_count += 1
    return inversion_count


def is_solvable_puzzle(puzzle_state):
    """
    Determines if a given n x n sliding puzzle is solvable based on the number of inversions and the blank tile's position.

    The solvability rules depend on the grid size (n):
    - For odd-sized grids (n is odd): The puzzle is solvable if and only if the number of inversions is even.
    - For even-sized grids (n is even): The puzzle is solvable if:
        a) The blank tile is on an even row from the bottom (rows are counted from 1 at the bottom) AND the number of inversions is odd.
        b) The blank tile is on an odd row from the bottom (rows are counted from 1 at the bottom) AND the number of inversions is even.
       This can be simplified to: solvable if (blank_row_from_bottom + inversions) is even.

    Args:
        puzzle_state (list[list[int]]): The puzzle state to check for solvability.

    Returns:
        bool: True if the puzzle is solvable, False otherwise.
    """
    inversions = count_inversions(puzzle_state)
    grid_dimension = len(puzzle_state)

    if grid_dimension % 2 == 1:  # Odd-sized grid: Solvable if inversions are even
        return inversions % 2 == 0
    else:  # Even-sized grid: Solvability depends on blank row from bottom and inversions
        blank_tile_row = get_empty_tile_pos(puzzle_state)[0]
        blank_row_from_bottom = (
            grid_dimension - blank_tile_row
        )  # Calculate blank tile's row index from the bottom (1-indexed)
        return (
            blank_row_from_bottom + inversions
        ) % 2 == 0  # Solvable if sum of blank row from bottom and inversions is even

In [6]:
def solvePuzzle(state, heuristic):
    """Solves the n**2-1 puzzle using the A* search algorithm.

    This function applies the A* search algorithm to find the optimal sequence of moves to solve a given n x n sliding puzzle.
    It utilizes a provided heuristic function to guide the search towards the goal state.

    Inputs:
        state (list[list[int]]): The initial state of the puzzle as a 2D list.
        heuristic (Callable): A heuristic function that estimates the cost from a given state to the goal state.
                              This should be one of the heuristic functions defined (e.g., misplaced_tiles_heuristic, manhattan_distance_heuristic, enhanced_heuristic).

    Outputs:
        tuple: A tuple containing the results of the search:
            - steps (int): The number of moves in the optimal solution path (excluding the initial state).
            - expanded_nodes (int): The total number of nodes expanded during the A* search process.
            - max_frontier_size (int): The maximum number of nodes that were in the search frontier at any point during the search.
            - optimal_path (Optional[list[list[list[int]]]]): The optimal path from the initial state to the goal state, represented as a list of puzzle states.
                                                               Returns None if the initial state is invalid.
            - error_code (int): An integer error code indicating the outcome of the search:
                0 for successful solution,
                -1 if the initial puzzle state is invalid (e.g., incorrect dimensions or tile values),
                -2 if the puzzle is determined to be unsolvable from the given initial state.
    """
    move_count, expanded_nodes, max_frontier_size = 0, 0, 0
    optimal_path = []

    if not is_valid_puzzle_state(state):  # Validate the initial puzzle state
        return (
            move_count,
            expanded_nodes,
            max_frontier_size,
            None,
            -1,
        )  # Error code -1: Invalid puzzle state
    if not is_solvable_puzzle(state):  # Check if the puzzle is solvable
        return (
            move_count,
            expanded_nodes,
            max_frontier_size,
            optimal_path,
            -2,
        )  # Error code -2: Unsolvable puzzle

    priority_queue = (
        PriorityQueue()
    )  # Initialize the priority queue for A* search frontier
    explored_states = (
        set()
    )  # Initialize a set to keep track of explored states to prevent cycles and redundant searches

    start_node = PuzzleNode(state, None, 0, heuristic)  # Create the initial PuzzleNode
    priority_queue.put(
        (start_node.total_cost_f, start_node)
    )  # Add the start node to the priority queue
    current_frontier_size = 1
    max_frontier_size = 1  # Initialize max frontier size to 1 (for the initial node)

    goal_state_config = create_goal_state(
        len(state)
    )  # Generate the target goal state for comparison

    while (
        not priority_queue.empty()
    ):  # A* search loop continues as long as there are nodes in the frontier
        _, current_node = (
            priority_queue.get()
        )  # Retrieve and remove the node with the lowest f-value from the frontier
        current_frontier_size -= 1  # Decrease frontier size as a node is removed

        current_puzzle_state_tuple = to_immutable_state(
            current_node.state
        )  # Convert state to tuple for hashability
        if (
            current_puzzle_state_tuple in explored_states
        ):  # Check if the current state has already been explored
            continue  # If explored, skip to the next iteration to avoid redundant processing

        explored_states.add(
            current_puzzle_state_tuple
        )  # Mark the current state as explored

        if (
            current_node.state == goal_state_config
        ):  # Check if the current node's state is the goal state
            solution_path_node = current_node
            while (
                solution_path_node
            ):  # Reconstruct the optimal path by tracing back through parent pointers
                optimal_path.append(
                    solution_path_node.state
                )  # Add each state to the optimal path list
                solution_path_node = (
                    solution_path_node.parent_node
                )  # Move to the parent node
            optimal_path.reverse()  # Reverse the path to get the correct order from start to goal
            move_count = (
                len(optimal_path) - 1
            )  # Calculate the number of moves (path length - 1)
            return (
                move_count,
                expanded_nodes,
                max_frontier_size,
                optimal_path,
                0,
            )  # Return success with error code 0

        expanded_nodes += 1  # Increment the count of expanded nodes
        successor_nodes = (
            current_node.get_successors()
        )  # Generate successor nodes (children) for the current node

        for successor in successor_nodes:  # Iterate through each successor node
            successor_state_tuple = to_immutable_state(
                successor.state
            )  # Convert successor state to tuple for checking exploration
            if (
                successor_state_tuple not in explored_states
            ):  # Check if the successor state has not been explored yet
                priority_queue.put(
                    (successor.total_cost_f, successor)
                )  # Add the successor node to the priority queue
                current_frontier_size += 1  # Increase current frontier size
                max_frontier_size = max(
                    max_frontier_size, current_frontier_size
                )  # Update max frontier size if current is larger

    return (
        move_count,
        expanded_nodes,
        max_frontier_size,
        optimal_path,
        -2,
    )  # Error code -2: Search failed to find a solution (should not be reached for solvable puzzles)

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [7]:
## Test for state not correctly defined

incorrect_state = [[0, 1, 2], [2, 3, 4], [5, 6, 7]]
_, _, _, _, err = solvePuzzle(incorrect_state, lambda state: 0)
assert err == -1

In [8]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = (
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[7, 0, 8], [4, 6, 1], [5, 3, 2]],
    [[5, 7, 6], [2, 4, 3], [8, 1, 0]],
)

# Test the values returned by the heuristic functions
h_mt_vals = [7, 8, 7]
h_man_vals = [15, 17, 18]

for i in range(0, 3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert h_mt == h_mt_vals[i]
    assert h_man == h_man_vals[i]

In [9]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[2, 3, 7], [1, 8, 4], [6, 5, 0]],
    [[2, 3, 7], [1, 8, 4], [6, 0, 5]],
    [[2, 3, 7], [1, 0, 4], [6, 8, 5]],
    [[2, 0, 7], [1, 3, 4], [6, 8, 5]],
    [[0, 2, 7], [1, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [0, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 0, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 4, 0], [6, 8, 5]],
    [[1, 2, 0], [3, 4, 7], [6, 8, 5]],
    [[1, 0, 2], [3, 4, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 0, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 0], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 5], [6, 8, 0]],
    [[1, 4, 2], [3, 7, 5], [6, 0, 8]],
    [[1, 4, 2], [3, 0, 5], [6, 7, 8]],
    [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
]

astar_steps = [17, 25, 28]
for i in range(0, 3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[0]
    )
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_mt == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert expansions_man < expansions_mt
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert opt_path_mt == opt_path_soln

In [10]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [
    [1, 2, 6, 3],
    [0, 9, 5, 7],
    [4, 13, 10, 11],
    [8, 12, 14, 15],
]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[0]
)
steps_man, expansions_man, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[1]
)
# Test whether the number of optimal steps is correct and the same
assert steps_mt == steps_man == 9
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert expansions_mt >= expansions_man

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [11]:
## Puzzle solvability test

unsolvable_initial_state = [[7, 5, 6], [2, 4, 3], [8, 1, 0]]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert err == -2

In [12]:
## Extra heuristic function test.
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0, 3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[2]
    )
    steps_man, expansions_man, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_new == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert dom > 0

In [13]:
## Memoization test - will be carried out after submission

**AI Statement**

I turned off GitHub Copilot Autocomplete and primarily used AI to transcribe my thoughts via Whisper (words) into clear strategy (plain text) #CS110-AlgoStratDataStruct style. My approach centered around a thorough understanding and implementation of the A* search algorithm to effectively solve the 8-puzzle problem. Before even beginning to write code, I invested significant time in researching the core principles of A*, exploring various heuristic functions, and gaining a solid grasp of the puzzle's challenges. The foundational knowledge from CS110, particularly the principles of algorithm design and strategic problem-solving, heavily influenced my methodology. While initially I felt uncertain about the most efficient coding strategies, my extensive research phase was crucial in clarifying these points and building a strong conceptual foundation before moving to implementation. I relied heavily on academic resources and explored different search strategies to ensure a robust and informed approach, ensuring I could implement a solution rooted in solid computer science principles rather than relying on automated code generation. During the coding phase, I made conscious decisions driven by my research and analytical evaluation. For example, in choosing a heuristic function, I started by considering the basic misplaced tiles and Manhattan distance heuristics. However, through further investigation, I opted to also implement a more sophisticated heuristic, incorporating linear conflict detection. My analysis indicated that while Manhattan distance is admissible, incorporating linear conflicts could lead to a more informed search and improve performance, particularly for more complex puzzle states. I also evaluated different options for managing the search frontier, ultimately deciding on a priority queue to efficiently prioritize nodes with the lowest estimated total cost.

**References Used**

Russell, S. J., & Norvig, P. (2016). Artificial intelligence: a modern approach (3rd ed.). Pearson Education.

Wikipedia contributors. (2023, November 28). A search algorithm. In Wikipedia, The Free Encyclopedia. Retrieved from https://en.wikipedia.org/wiki/A*_search_algorithm*.

Nilsson, N. J. (1998). Artificial intelligence: a new synthesis. Morgan Kaufmann. 

> Utilized this Classic AI textbook for detailed coverage of search algorithms.

Culberson, J. C., & Schaeffer, J. (1998). Pattern databases. Computational Intelligence, 14(3), 318-334. 
> Pioneering paper on Pattern Databases for heuristic search, relevant to my studies for h3.

Hansson, O., Mayer, A., & Yung, M. (1992). Criticizing solutions to relaxed models yields powerful admissible heuristics. Information Processing Letters, 44(4), 183-190. 
> Paper discussing admissible heuristics and techniques that inspired linear conflict heuristic.